In [1]:
# ACE-Step API Demo
# ==================
# This notebook demonstrates the ACE-Step music API endpoints using
# the MP3 files in this directory: birb.mp3, random_song.mp3, wotah.mp3
#
# Prerequisites:
#   - ACE-Step API server running
#   - pip install requests

import requests
import json
import time
from pathlib import Path

BASE_URL = "https://abox-noneruptive-felisha.ngrok-free.dev"
AUDIO_DIR = Path(".")  # MP3 files are in the same directory as this notebook

# Ngrok free tier requires this header to bypass the browser interstitial page.
SESSION = requests.Session()
SESSION.headers.update({"ngrok-skip-browser-warning": "true"})
SESSION.auth = ("admin", "goldenhands")  # Basic Auth for ngrok

# List available audio files
mp3_files = sorted(AUDIO_DIR.glob("*.mp3"))
print("Available MP3 files:")
for f in mp3_files:
    print(f"  - {f.name}")

Available MP3 files:
  - birb.mp3
  - california.mp3
  - random_song.mp3
  - wotah.mp3


## 1. Health Check
# Verify the API server is running before making other calls.

In [2]:
resp = SESSION.get(f"{BASE_URL}/health")
print(json.dumps(resp.json(), indent=2))

{
  "data": {
    "status": "ok",
    "service": "ACE-Step API",
    "version": "1.0"
  },
  "code": 200,
  "error": null,
  "timestamp": 1770542189896,
  "extra": null
}


## 2. List Available Models & Server Stats

In [3]:
# List available DiT models
models_resp = SESSION.get(f"{BASE_URL}/v1/models")
print("=== Available Models ===")
print(json.dumps(models_resp.json(), indent=2))

# Server statistics
stats_resp = SESSION.get(f"{BASE_URL}/v1/stats")
print("\n=== Server Stats ===")
print(json.dumps(stats_resp.json(), indent=2))

=== Available Models ===
{
  "data": {
    "models": [
      {
        "name": "acestep-v15-turbo",
        "is_default": true
      }
    ],
    "default_model": "acestep-v15-turbo"
  },
  "code": 200,
  "error": null,
  "timestamp": 1770542192388,
  "extra": null
}

=== Server Stats ===
{
  "data": {
    "jobs": {
      "total": 0,
      "queued": 0,
      "running": 0,
      "succeeded": 0,
      "failed": 0
    },
    "queue_size": 0,
    "queue_maxsize": 200,
    "avg_job_seconds": 5.0
  },
  "code": 200,
  "error": null,
  "timestamp": 1770542192472,
  "extra": null
}


## 3. LM Understand Audio (`/lm/understand`)
Analyze each MP3 file to extract caption, lyrics, BPM, key, duration, and other metadata.
This is the reverse of generation: audio in, structured description out.

In [4]:
# Analyze birb.mp3
with open(AUDIO_DIR / "birb.mp3", "rb") as f:
    resp = SESSION.post(
        f"{BASE_URL}/lm/understand",
        files={"audio": ("birb.mp3", f, "audio/mpeg")},
        data={"temperature": 0.3},
    )

print("=== birb.mp3 analysis ===")
print(json.dumps(resp.json(), indent=2))

=== birb.mp3 analysis ===
{
  "data": {
    "caption": "A clean, dry studio recording of a standard acoustic drum kit. The clip begins with a brief hi-hat count-in followed by a straightforward, punchy rock groove. The kick on the downbeats and a crisp snare on the backbeats establish the rhythm, while the hi-hats maintain a steady pulse. A short tom and cymbal fill transitions the loop into a final, punctuated crash cymbal hit.",
    "lyrics": "[Instrumental]",
    "bpm": 167,
    "duration": 11.0,
    "key_scale": "C# minor",
    "language": "unknown",
    "time_signature": "4",
    "seed": null
  },
  "code": 200,
  "error": null,
  "timestamp": 1770542197990,
  "extra": null
}


In [5]:
# Analyze california.mp3
with open(AUDIO_DIR / "california.mp3", "rb") as f:
    resp = SESSION.post(
        f"{BASE_URL}/lm/understand",
        files={"audio": ("california.mp3", f, "audio/mpeg")},
        data={"temperature": 0.3},
    )

print("=== california.mp3 analysis ===")
print(json.dumps(resp.json(), indent=2))

=== california.mp3 analysis ===
{
  "data": {
    "caption": "A classic blues-rock track driven by a dual electric guitar attack. The song opens with an extended, expressive guitar intro featuring melodic, blues-infused licks and bends over a steady rock groove from the bass and drums. A clear, slightly gritty male vocal enters, delivering a narrative in a straightforward rock style. The arrangement is built around a standard verse-chorus structure, punctuated by intricate guitar fills. The track culminates in a fiery, extended guitar solo filled with rapid-fire licks, string bends, and a classic overdriven tone before a final, decisive chord ends the song.",
    "lyrics": "[Intro - Guitar Riff]\n\n[Verse 1]\nEver got a feeling\nEverything's alright\n'Til an unexpected turn\nAnd another force in your sight\nLike when you're stripping your blankets\nAnd you're sweeping up the pole\nAnd all you thought you knew about the lady\nNow she's out of your hole\n\n[Verse 2]\nEver got a feeling\n

In [7]:
# Analyze wotah.mp3
with open(AUDIO_DIR / "wotah.mp3", "rb") as f:
    resp = SESSION.post(
        f"{BASE_URL}/lm/understand",
        files={"audio": ("wotah.mp3", f, "audio/mpeg")},
        data={"temperature": 0.3},
    )

print("=== wotah.mp3 analysis ===")
print(json.dumps(resp.json(), indent=2))

=== wotah.mp3 analysis ===
{
  "data": {
    "caption": "A crisp, live acoustic drum kit lays down a straightforward and driving rock groove. The performance begins with a brief cymbal and tom fill before locking into a steady 4/4 beat. The hi-hats maintain a consistent eighth-note pattern, while the snare and kick drum provide a solid backbeat. The recording has a natural, roomy ambiance, giving the drums a live and authentic feel.",
    "lyrics": "[Drum beat starts]\n[Instrumental]",
    "bpm": 188,
    "duration": 11.0,
    "key_scale": "F# major",
    "language": "unknown",
    "time_signature": "4",
    "seed": null
  },
  "code": 200,
  "error": null,
  "timestamp": 1770542410523,
  "extra": null
}


## 4. LM Inspire (`/lm/inspire`)
Generate caption, lyrics, and full metadata from a natural language description.
No audio is produced -- just the creative "blueprint" for a song.

In [8]:
# Inspire: vocal song
resp = SESSION.post(
    f"{BASE_URL}/lm/inspire",
    json={
        "query": "a chill lo-fi hip hop beat with soft piano and rain sounds, perfect for late-night studying",
        "temperature": 0.85,
    },
)

print("=== Inspire (vocal) ===")
print(json.dumps(resp.json(), indent=2))

=== Inspire (vocal) ===
{
  "data": {
    "caption": "A chill, atmospheric lo-fi hip-hop track built on a foundation of a steady, relaxed drum machine beat and a smooth, round synth bass. The core melodic element is a gentle, slightly melancholic piano loop, complemented by airy synth pads that create a spacious, dreamy soundscape. The arrangement evolves with the introduction of a clean, melodic synth lead and subtle arpeggiated textures. The track uses transitions smoothly marked by reverse cymbal swells and brief breakdowns where the beat drops out, enhancing its introspective and calming mood.",
    "lyrics": "[Instrumental]",
    "bpm": 91,
    "duration": 182.0,
    "key_scale": "B\u266d major",
    "language": "unknown",
    "time_signature": "4",
    "instrumental": false,
    "seed": null
  },
  "code": 200,
  "error": null,
  "timestamp": 1770542423373,
  "extra": null
}


In [ ]:
# Inspire: instrumental
resp = SESSION.post(
    f"{BASE_URL}/lm/inspire",
    json={
        "query": "energetic electronic dance music with heavy bass drops and synth arpeggios",
        "instrumental": True,
        "temperature": 0.9,
        "seed": 42,
    },
)

print("=== Inspire (instrumental) ===")
print(json.dumps(resp.json(), indent=2))

## 5. LM Format (`/lm/format`)
Enhance rough caption/lyrics into a polished, structured form with full metadata.

In [9]:
# Format: enhance rough input
resp = SESSION.post(
    f"{BASE_URL}/lm/format",
    json={
        "prompt": "sad acoustic ballad",
        "lyrics": "I walk alone tonight\nthe stars don't shine so bright\nwithout you here beside me\nnothing feels alright",
        "language": "en",
    },
)

print("=== Format (enhanced) ===")
print(json.dumps(resp.json(), indent=2))

=== Format (enhanced) ===
{
  "data": {
    "caption": "A sparse and melancholic acoustic ballad featuring a lone male vocalist and a fingerpicked acoustic guitar. The guitar plays a clean, arpeggiated chord progression with a touch of natural room reverb, creating an intimate and lonely atmosphere. The male vocal is gentle, breathy, and emotional, delivered with a sense of longing and sadness. The production is minimal and raw, focusing entirely on the heartfelt performance and the simple, poignant melody.",
    "lyrics": "[Intro - Acoustic Guitar]\n\n[Verse 1]\nI walk alone tonight\nThe stars don't shine so bright\nWithout you here beside me\nNothing feels alright\n\n[Outro]\nAlright\nAlright\nAlright",
    "bpm": 67,
    "duration": 60.0,
    "key_scale": "C major",
    "language": "",
    "time_signature": "2",
    "seed": null
  },
  "code": 200,
  "error": null,
  "timestamp": 1770542437308,
  "extra": null
}


In [ ]:
# Format: with metadata constraints
resp = SESSION.post(
    f"{BASE_URL}/lm/format",
    json={
        "prompt": "funk groove",
        "lyrics": "[Verse]\nGet up and move your body\nFeel the rhythm take control",
        "bpm": 110,
        "key_scale": "E Minor",
        "time_signature": "4",
        "duration": 180,
    },
)

print("=== Format (constrained) ===")
print(json.dumps(resp.json(), indent=2))

## 6. Music Generation (`/release_task` + `/query_result`)
Submit an async generation task, then poll until the result is ready.

In [11]:
def wait_for_task(task_id, poll_interval=3, max_wait=300):
    """Poll /query_result until the task succeeds or fails."""
    elapsed = 0
    while elapsed < max_wait:
        resp = SESSION.post(
            f"{BASE_URL}/query_result",
            json={"task_id_list": [task_id]},
        )
        data = resp.json()["data"]
        if data:
            task = data[0]
            status = task["status"]
            if status == 1:
                print(f"Task {task_id} succeeded after ~{elapsed}s")
                return task
            elif status == 2:
                print(f"Task {task_id} FAILED after ~{elapsed}s")
                return task
            else:
                print(f"  [{elapsed}s] status={status} (queued/running)...")
        time.sleep(poll_interval)
        elapsed += poll_interval
    print(f"Task {task_id} timed out after {max_wait}s")
    return None

In [ ]:
# 6a. Basic text-to-music generation
resp = SESSION.post(
    f"{BASE_URL}/release_task",
    json={
        "prompt": "upbeat indie pop with jangly guitars and handclaps",
        "lyrics": "[Verse 1]\nWake up, the sun is rising\nOpen up your eyes\nThe world outside is calling\nUnderneath the skies\n\n[Chorus]\nWe're running wild and free\nNothing's gonna stop you and me",
        "audio_duration": 30,
        "inference_steps": 8,
        "batch_size": 1,
    },
)

result = resp.json()
print(json.dumps(result, indent=2))
task_id = result["data"]["task_id"]
print(f"\nTask ID: {task_id}")

In [ ]:
# 6b. Poll for the result
task_result = wait_for_task(task_id)

if task_result and task_result["status"] == 1:
    result_data = json.loads(task_result["result"])
    for i, item in enumerate(result_data):
        print(f"\n--- Output {i} ---")
        print(f"  Audio URL : {BASE_URL}{item['file']}")
        print(f"  Prompt    : {item.get('prompt', '')[:80]}")
        print(f"  Metas     : {json.dumps(item.get('metas', {}))}")
        print(f"  Seed      : {item.get('seed_value', '')}")
        print(f"  DiT model : {item.get('dit_model', '')}")

## 7. Description-Driven Generation (`sample_query`)
Let the LM auto-generate caption, lyrics, and metadata from a plain text description.

In [12]:
# Description-driven generation using sample_query + thinking
resp = SESSION.post(
    f"{BASE_URL}/release_task",
    json={
        "sample_query": "a dreamy synthwave track with retro 80s vibes and vocoder vocals",
        "thinking": True,
        "audio_duration": 30,
        "batch_size": 1,
    },
)

result = resp.json()
print(json.dumps(result, indent=2))
task_id_sample = result["data"]["task_id"]

# Poll for result
task_result = wait_for_task(task_id_sample)
if task_result and task_result["status"] == 1:
    result_data = json.loads(task_result["result"])
    for i, item in enumerate(result_data):
        print(f"\n--- Output {i} ---")
        print(f"  Audio URL : {BASE_URL}{item['file']}")
        print(f"  Prompt    : {item.get('prompt', '')[:100]}")
        print(f"  Lyrics    : {item.get('lyrics', '')[:100]}...")

{
  "data": {
    "task_id": "2de207a6-7115-4384-866f-c5a802d87ca2",
    "status": "queued",
    "queue_position": 1
  },
  "code": 200,
  "error": null,
  "timestamp": 1770542464395,
  "extra": null
}
  [0s] status=0 (queued/running)...
  [3s] status=0 (queued/running)...
  [6s] status=0 (queued/running)...
  [9s] status=0 (queued/running)...
  [12s] status=0 (queued/running)...
  [15s] status=0 (queued/running)...
  [18s] status=0 (queued/running)...
  [21s] status=0 (queued/running)...
  [24s] status=0 (queued/running)...
  [27s] status=0 (queued/running)...
  [30s] status=0 (queued/running)...
  [33s] status=0 (queued/running)...
  [36s] status=0 (queued/running)...
  [39s] status=0 (queued/running)...
  [42s] status=0 (queued/running)...
Task 2de207a6-7115-4384-866f-c5a802d87ca2 succeeded after ~45s

--- Output 0 ---
  Audio URL : https://abox-noneruptive-felisha.ngrok-free.dev/v1/audio?path=%2Fhome%2Fkorcadav%2Fcompute%2Face_step%2F.cache%2Facestep%2Ftmp%2Fapi_audio%2Fc6f81e08-ad

## 8. Repaint / Remix (upload source audio)
Upload one of our MP3 files and repaint a section of it with a new style.

In [ ]:
# Repaint the first 15 seconds of random_song.mp3 with a jazz style
with open(AUDIO_DIR / "random_song.mp3", "rb") as f:
    resp = SESSION.post(
        f"{BASE_URL}/release_task",
        files={"src_audio": ("random_song.mp3", f, "audio/mpeg")},
        data={
            "prompt": "smooth jazz piano trio with brushed drums",
            "task_type": "repaint",
            "repainting_start": 0.0,
            "repainting_end": 15.0,
            "inference_steps": 8,
            "batch_size": 1,
        },
    )

result = resp.json()
print(json.dumps(result, indent=2))

if result.get("data", {}).get("task_id"):
    task_id_repaint = result["data"]["task_id"]
    task_result = wait_for_task(task_id_repaint)
    if task_result and task_result["status"] == 1:
        result_data = json.loads(task_result["result"])
        for item in result_data:
            print(f"  Repainted audio: {BASE_URL}{item['file']}")

## 9. Style Transfer (reference audio)
Use one MP3 as a style reference to influence the generation of new music.

In [ ]:
# Use wotah.mp3 as a style reference for new generation
with open(AUDIO_DIR / "wotah.mp3", "rb") as f:
    resp = SESSION.post(
        f"{BASE_URL}/release_task",
        files={"ref_audio": ("wotah.mp3", f, "audio/mpeg")},
        data={
            "prompt": "ambient electronic with ethereal pads",
            "lyrics": "[Verse]\nFloating through the cosmos\nStars illuminate the void",
            "audio_cover_strength": 0.2,  # low strength = subtle style transfer
            "audio_duration": 30,
            "inference_steps": 8,
            "batch_size": 1,
        },
    )

result = resp.json()
print(json.dumps(result, indent=2))

if result.get("data", {}).get("task_id"):
    task_id_style = result["data"]["task_id"]
    task_result = wait_for_task(task_id_style)
    if task_result and task_result["status"] == 1:
        result_data = json.loads(task_result["result"])
        for item in result_data:
            print(f"  Style-transferred audio: {BASE_URL}{item['file']}")

## 10. Download Generated Audio
Download a generated audio file to disk using the `/v1/audio` endpoint.

In [ ]:
def download_audio(audio_url, output_path):
    """Download an audio file from the API.
    
    audio_url: full URL like http://host/v1/audio?path=...
    """
    resp = SESSION.get(audio_url)
    if resp.status_code == 200:
        Path(output_path).write_bytes(resp.content)
        size_kb = len(resp.content) / 1024
        print(f"Downloaded {output_path} ({size_kb:.1f} KB)")
    else:
        print(f"Download failed: {resp.status_code} - {resp.text[:200]}")

# Example: download the first generated audio from section 6 (if it ran successfully)
# Uncomment and replace with an actual URL from a completed task:
# download_audio(f"{BASE_URL}/v1/audio?path=<path_from_result>", "generated_output.mp3")